In [ ]:
!pip install google
!pip install --upgrade google-cloud-vision

In [ ]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    # texts = response.text_annotations
    # print('Texts:')

    # for text in texts:
    #     print('\n"{}"'.format(text.description))

    #     vertices = (['({},{})'.format(vertex.x, vertex.y)
    #                 for vertex in text.bounding_poly.vertices])

    #     print('bounds: {}'.format(','.join(vertices)))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    else:
        return response

In [ ]:
response = detect_text("/home/josel/Git/Bill/SCR-20230516-sdja.png")

In [ ]:
res = str(response.text_annotations[0].description).splitlines()

res = list(map(lambda x: x.strip(),res))

char = list(map(lambda x: x[0],res))

res.append('Por favor responde apenas com uma letra.')

res = '\n'.join(res)

print(res)

In [ ]:
import os
import openai

openai.api_key=""

def pergunta(text):

    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": text}
    ]
    )

    return completion.choices[0].message.content

In [ ]:
print(pergunta(res))

In [ ]:
def resposta(image):
    response = detect_text(image)
    res = str(response.text_annotations[0].description).splitlines()
    res = list(map(lambda x: x.strip(),res))
    res.append('Por favor responde apenas com uma letra.')

    res = '\n'.join(res)

    return pergunta(res)

In [ ]:
resposta("SCR-20230516-shrv.png")

In [ ]:
import os
import glob

# specify your path here
path = ''

# find all .png files in the specified directory
png_files = glob.glob(os.path.join(path, "*.png"))

# get the most recent file
latest_file = max(png_files, key=os.path.getmtime)

print(latest_file)


In [ ]:
import requests
import re

import asyncio
import aiohttp

async def make_request(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            response_text = await response.text()
            return response_text

async def description(query):

    if not re.search(r'(?:QUAL +D(?:O|A)S +SEGUINTES)|(?:QUAL +DESTES)',query):

        url = "https://duckduckgo8.p.rapidapi.com/"

        querystring = {"q":query}

        headers = {
            "X-RapidAPI-Key": "",
            "X-RapidAPI-Host": "duckduckgo8.p.rapidapi.com"
        }

        async with aiohttp.ClientSession() as session:
            async with session.get(url, headers=headers, params=querystring) as response:
                res = await response.json()
                return '\n\n'.join([res['results'][0]['description'],res['results'][1]['description'],res['results'][2]['description']])
    else:
        return None

In [ ]:
import asyncio

from google.cloud import vision
import io

import re

import logging

# Set up logging configuration
logging.basicConfig(
    level=logging.INFO,
    filename='app.log',
    filemode='w',
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def detect_text(path):
    """Detects text in the file."""
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    # texts = response.text_annotations
    # print('Texts:')

    # for text in texts:
    #     print('\n"{}"'.format(text.description))

    #     vertices = (['({},{})'.format(vertex.x, vertex.y)
    #                 for vertex in text.bounding_poly.vertices])

    #     print('bounds: {}'.format(','.join(vertices)))

    if response.error.message:
        logging.info(f'Google Text Failed')
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    else:
        # logging.info(f'Google Text: {response.text_annotations[0].description}')
        return response
    

import os
import openai

openai.api_key=""

async def pergunta(text):

    completion = await openai.ChatCompletion.acreate(
    model="gpt-3.5-turbo",
    temperature=0.5,
    messages=[
        {"role": "user", "content": text}
    ],
    timeout=4
    )

    return completion.choices[0].message.content

def resposta(image):
    response = detect_text(image)
    res = str(response.text_annotations[0].description).splitlines()
    res = list(map(lambda x: x.strip(),res))

    for line in res[:5]:
        if len(line) <= 6:
            res.pop(0)
        else:
            break

    j = 0

    for i,line in enumerate(res[:3]):
        if '?' in line:
            j=i
            break

    for _ in range(j):
        line = res.pop(1)
        res[0] = res[0] + ' ' + line

    for i, line in enumerate(res[1:]):
        if re.search(r'.*D(?:r|e)(?:\.|:)Why.*',line):
            res = res[:i+1]
            break

    res.append('Por favor responde apenas com a letra correspondente à resposta (se não tiver letra considere a primeira opcão a letra A, segunda a B ...) e a resposta.\nSe não tiver opções simplesmente responda sucitamente.')

    first = res[0]

    res = '\n'.join(res)

    return res,first



async def resposta_final(image,loop):
    res,first = resposta(image)

    logging.info('Google Texto: '+res)

    task1 = loop.create_task(description(first), name='description')

    task2 = loop.create_task(pergunta(res), name='pergunta')

    l = [task1,task2]

    while True:
        done, pending = await asyncio.wait(l, return_when=asyncio.FIRST_COMPLETED)

        # Process completed tasks
        for task in done:
            l.remove(task)

            if task.get_name() == 'description':
                logging.info("Descricao: "+task.result())
            elif task.get_name() == 'pergunta':
                logging.info("Resposta: "+task.result())

        if not pending:
            break

In [ ]:
import time

# async def main():

#     loop = asyncio.get_event_loop()

#     for i in range(1):
#         path = f'images/IMG_EF1F1263C7AA-{i+1}.jpeg'
#         await resposta_final(path,loop)
#         time.sleep(1)

# await main()
from telegram import Update
from telegram.ext.dispatcher import run_async
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# Replace YOUR_API_TOKEN with your actual bot API token
API_TOKEN = ""

# Define a start command handler
def start(update: Update, context: CallbackContext):
    update.message.reply_text("Send me an image and I'll process it.")


# Define the function to process the image
def process_image(image_file_id):
    print("Image file ID:", image_file_id)
    # Add your image processing logic here
    return "Image processing done."




async def image_handler_async(update: Update, context: CallbackContext):

    logging.info('Entering image_handler')

    loop = asyncio.get_event_loop()

    # get the image file
    image = update.message.photo[-1].get_file()

    # download the image
    image.download('image.jpg')  # replace 'image.jpg' with your preferred file path and name

    # send a response message

    res,first = resposta('image.jpg')

    task1 = loop.create_task(description(first), name='description')

    task2 = loop.create_task(pergunta(res), name='pergunta')

    l = [task1,task2]

    while True:
        done, pending = await asyncio.wait(l, return_when=asyncio.FIRST_COMPLETED)

        # Process completed tasks
        for task in done:
            l.remove(task)

            if task.get_name() == 'description':
                logging.info("Descricao: "+task.result())
            elif task.get_name() == 'pergunta':
                logging.info("Resposta: "+task.result())
            update.message.reply_text(task.result())

        if not pending:
            break

def image_handler(update: Update, context: CallbackContext):
    asyncio.run(image_handler_async(update,context))



async def main():
    updater = Updater(API_TOKEN)

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # Register the handlers
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(MessageHandler(Filters.photo, image_handler))

    # Start the bot
    updater.start_polling()

    # Run the bot until Ctrl-C is pressed or the process receives SIGINT, SIGTERM or SIGABRT
    updater.idle()


if __name__ == "__main__":
    await main()

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage you want to scrape
url = 'https://brasilescola.uol.com.br/geografia/asia.htm'

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find the desired elements or tags containing the text
# For example, if you want to scrape all the text within <p> tags:
paragraphs = soup.find_all('p')

# Extract the text from the elements
for paragraph in paragraphs:
    text = paragraph.get_text()
    print(text)